# Notebook Setup
<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Generative-AI-Integration-Patterns-1E/blob/main/02-Batch-Summarization/Chapter_6_Integration_pattern_batch_summarization.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
#Install dependencies

!pip install --upgrade google-cloud-aiplatform

In [ ]:
#Authenticate
!gcloud auth application-default login

In [ ]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

In [ ]:
PROJECT = "YOUR-GCP-PROJECT"#@param {type:"string"}
LOCATION = "us-central1"#@param {type:"string"}
MODEL = "gemini-1.5-pro-001"#@param {type:"string"}

# Function Definitions

In [ ]:
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0,
    "top_p": 0.95,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

def generate(prompt_template_beggining,document,prompt_template_end):
  vertexai.init(project=PROJECT, location=LOCATION)
  model = GenerativeModel(MODEL)
  responses = model.generate_content(
      [prompt_template_beggining,document,prompt_template_end],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=False,
  )
  return(responses)


# Entry Point

In [ ]:
#@title Example signup forms
JSON_DOC = """
[
    {
        "company_information": {
            "legal_name": "Tech Solutions Inc.",
            "dba_name": "TechSol",
            "company_type": "Corporation",
            "industry": "Technology",
            "industry_description": "Software development and IT consulting services.",
            "website": "https://www.techsolutions.com",
            "date_founded": "2010-03-15"
        },
        "business_details": {
            "expected_transaction_volume": 5000,
            "average_transaction_value": 120,
            "business_model_description": "B2B software subscriptions and project-based consulting fees."
        },
        "contact_information": {
            "primary_contact": {
                "name": "John Smith",
                "email": "john.smith@techsolutions.com",
                "phone_number": "555-123-4567"
            },
            "business_address": {
                "street_address": "123 Main St",
                "city": "Anytown",
                "state": "CA",
                "postal_code": "91234",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "12-3456789",
            "aml_kyc_policy": "We adhere to strict AML/KYC regulations and conduct thorough customer due diligence."
        }
    },
    {
        "company_information": {
            "legal_name": "Acme Fintech Solutions Inc.",
            "dba_name": "Acme Pay",
            "company_type": "Corporation",
            "industry": "Financial Technology",
            "industry_description": "Developing a mobile payment platform for peer-to-peer transactions.",
            "website": "https://www.acmepay.com",
            "date_founded": "2021-03-15"
        },
        "business_details": {
            "expected_transaction_volume": 50000,
            "average_transaction_value": 75,
            "business_model_description": "We charge a small fee on each transaction processed through our platform."
        },
        "contact_information": {
            "primary_contact": {
                "name": "John Smith",
                "email": "john.smith@acmepay.com",
                "phone_number": "555-123-4567"
            },
            "business_address": {
                "street_address": "123 Main Street",
                "city": "San Francisco",
                "state": "CA",
                "postal_code": "94105",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "12-3456789",
            "aml_kyc_policy": "We have a robust AML/KYC policy in place, including identity verification and transaction monitoring."
        }
    },
    {
        "company_information": {
            "legal_name": "Green Thumbs LLC",
            "dba_name": "Green Thumbs",
            "company_type": "LLC",
            "industry": "E-commerce",
            "industry_description": "Online marketplace for CBD and hemp gardening supplies.",
            "website": "https://www.greenthumbs.com",
            "date_founded": "2022-08-01"
        },
        "business_details": {
            "expected_transaction_volume": 2000,
            "average_transaction_value": 120,
            "business_model_description": "We earn a commission on each sale made through our platform."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Jane Doe",
                "email": "jane.doe@greenthumbs.com",
                "phone_number": "555-234-5678"
            },
            "business_address": {
                "street_address": "456 Oak Avenue",
                "city": "Los Angeles",
                "state": "CA",
                "postal_code": "90001",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "98-7654321",
            "aml_kyc_policy": "We comply with all applicable AML/KYC regulations."
        }
    },
    {
        "company_information": {
            "legal_name": "Tech Gun Gear Inc.",
            "dba_name": "Tech Gun Gear",
            "company_type": "Corporation",
            "industry": "Retail",
            "industry_description": "Retailer of guns, hand weapons and gadgets.",
            "website": "https://www.techgear.com",
            "date_founded": "2019-01-15"
        },
        "business_details": {
            "expected_transaction_volume": 10000,
            "average_transaction_value": 250,
            "business_model_description": "We sell firearms directly to consumers through our online store and physical locations."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Robert Lee",
                "email": "robert.lee@techgear.com",
                "phone_number": "555-345-6789"
            },
            "business_address": {
                "street_address": "789 Pine Street",
                "city": "New York",
                "state": "NY",
                "postal_code": "10001",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "55-6677889",
            "aml_kyc_policy": "What is KYC?"
        }
    },
    {
        "company_information": {
            "legal_name": "Travel Adventures LLC",
            "dba_name": "Travel Adventures",
            "company_type": "LLC",
            "industry": "Travel",
            "industry_description": "Online travel booking platform for adventure travel experiences.",
            "website": "https://www.traveladventures.com",
            "date_founded": "2020-07-01"
        },
        "business_details": {
            "expected_transaction_volume": 1500,
            "average_transaction_value": 1500,
            "business_model_description": "We earn a commission on each travel booking made through our platform."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Maria Garcia",
                "email": "maria.garcia@traveladventures.com",
                "phone_number": "555-456-7890"
            },
            "business_address": {
                "street_address": "312 Cedar Street",
                "city": "Miami",
                "state": "FL",
                "postal_code": "33133",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "33-4455667",
            "aml_kyc_policy": "We adhere to strict AML/KYC guidelines."
        }
    },
    {
        "company_information": {
            "legal_name": "Creative Designs Inc.",
            "dba_name": "Creative Designs",
            "company_type": "Corporation",
            "industry": "Design Services",
            "industry_description": "Providing graphic design and branding services to businesses.",
            "website": "https://www.creativedesigns.com",
            "date_founded": "2018-05-10"
        },
        "business_details": {
            "expected_transaction_volume": 500,
            "average_transaction_value": 5000,
            "business_model_description": "We offer design packages and hourly rates for our services."
        },
        "contact_information": {
            "primary_contact": {
                "name": "David Chen",
                "email": "david.chen@creativedesigns.com",
                "phone_number": "555-567-8901"
            },
            "business_address": {
                "street_address": "214 Oakwood Lane",
                "city": "Chicago",
                "state": "IL",
                "postal_code": "60606",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "22-3344556",
            "aml_kyc_policy": "We have established AML/KYC procedures to ensure compliance."
        }
    },
    {
        "company_information": {
            "legal_name": "Acme Robotics Inc.",
            "dba_name": "Acme Robotics",
            "company_type": "Corporation",
            "industry": "Robotics",
            "industry_description": "Developing and manufacturing cutting-edge robotic solutions for industrial automation.",
            "website": "https://www.acmerobotics.com",
            "date_founded": "2018-03-15"
        },
        "business_details": {
            "expected_transaction_volume": 500,
            "average_transaction_value": 10000,
            "business_model_description": "We sell robots and provide maintenance services."
        },
        "contact_information": {
            "primary_contact": {
                "name": "John Smith",
                "email": "john.smith@acmerobotics.com",
                "phone_number": "+1-555-123-4567"
            },
            "business_address": {
                "street_address": "123 Main St",
                "city": "Anytown",
                "state": "CA",
                "postal_code": "91234",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "12-3456789",
            "aml_kyc_policy": "We adhere to strict AML/KYC regulations and conduct thorough customer due diligence."
        }
    },
    {
        "company_information": {
            "legal_name": "Green Thumb Farms LLC",
            "dba_name": "Green Thumb Farms",
            "company_type": "LLC",
            "industry": "Agriculture",
            "industry_description": "Organic farming of fruits and vegetables for local markets and restaurants.",
            "website": "https://www.greenthumbfarms.com",
            "date_founded": "2015-07-22"
        },
        "business_details": {
            "expected_transaction_volume": 200,
            "average_transaction_value": 500,
            "business_model_description": "Direct-to-consumer sales through farmers markets and online orders, as well as wholesale to restaurants."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Jane Doe",
                "email": "jane.doe@greenthumbfarms.com",
                "phone_number": "+1-555-234-5678"
            },
            "business_address": {
                "street_address": "456 Oak Ave",
                "city": "Springfield",
                "state": "IL",
                "postal_code": "60652",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "98-7654321",
            "aml_kyc_policy": "We comply with all applicable AML/KYC regulations and verify customer identities."
        }
    },
    {
        "company_information": {
            "legal_name": "Creative Designs Co.",
            "dba_name": "Creative Designs",
            "company_type": "Partnership",
            "industry": "Design Services",
            "industry_description": "Providing graphic design, web design, and branding services to small businesses.",
            "website": "https://www.creativedesigns.co",
            "date_founded": "2020-01-10"
        },
        "business_details": {
            "expected_transaction_volume": 100,
            "average_transaction_value": 2500,
            "business_model_description": "Project-based work with upfront payments and recurring maintenance contracts."
        },
        "contact_information": {
            "primary_contact": {
                "name": "David Lee",
                "email": "david.lee@creativedesigns.co",
                "phone_number": "+1-555-345-6789"
            },
            "business_address": {
                "street_address": "789 Pine St",
                "city": "Denver",
                "state": "CO",
                "postal_code": "80202",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "56-7890123",
            "aml_kyc_policy": "We follow industry best practices for AML/KYC compliance and customer verification."
        }
    },
    {
        "company_information": {
            "legal_name": "Sunrise Coffee Shop",
            "dba_name": "Sunrise Coffee",
            "company_type": "Sole Proprietorship",
            "industry": "Food & Beverage",
            "industry_description": "Serving specialty coffee, pastries, and light breakfast and lunch options.",
            "website": "https://www.sunrisecoffee.com",
            "date_founded": "2022-09-05"
        },
        "business_details": {
            "expected_transaction_volume": 1000,
            "average_transaction_value": 15,
            "business_model_description": "Retail sales of coffee and food items with a focus on customer experience and community building."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Sarah Kim",
                "email": "sarah.kim@sunrisecoffee.com",
                "phone_number": "+1-555-456-7890"
            },
            "business_address": {
                "street_address": "313 Maple Ave",
                "city": "Austin",
                "state": "TX",
                "postal_code": "78701",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "45-6789012",
            "aml_kyc_policy": "We comply with relevant AML/KYC regulations and have procedures for customer identification."
        }
    },
    {
        "company_information": {
            "legal_name": "Travel Adventures Inc.",
            "dba_name": "Travel Adventures",
            "company_type": "Corporation",
            "industry": "Travel & Tourism",
            "industry_description": "Providing customized travel planning and booking services for individuals and groups.",
            "website": "https://www.traveladventures.com",
            "date_founded": "2017-06-18"
        },
        "business_details": {
            "expected_transaction_volume": 75,
            "average_transaction_value": 5000,
            "business_model_description": "Fee-based travel planning and commission-based bookings of flights, accommodations, and activities."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Michael Chen",
                "email": "michael.chen@traveladventures.com",
                "phone_number": "+1-555-567-8901"
            },
            "business_address": {
                "street_address": "202 Cedar St",
                "city": "Seattle",
                "state": "WA",
                "postal_code": "98101",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "34-5678901",
            "aml_kyc_policy": "We have robust AML/KYC policies in place to prevent financial crime and ensure compliance."
        }
    },
    {
        "company_information": {
            "legal_name": "Tech Solutions Group LLC",
            "dba_name": "Tech Solutions Group",
            "company_type": "LLC",
            "industry": "Information Technology",
            "industry_description": "Offering IT consulting, software development, and network management services.",
            "website": "https://www.techsolutionsgroup.com",
            "date_founded": "2016-04-25"
        },
        "business_details": {
            "expected_transaction_volume": 300,
            "average_transaction_value": 1000,
            "business_model_description": "Hourly consulting fees, project-based contracts, and managed service agreements."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Emily Wilson",
                "email": "emily.wilson@techsolutionsgroup.com",
                "phone_number": "+1-555-678-9012"
            },
            "business_address": {
                "street_address": "999 Birch St",
                "city": "San Francisco",
                "state": "CA",
                "postal_code": "94102",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "23-4567890",
            "aml_kyc_policy": "We maintain strict adherence to AML/KYC regulations and perform thorough customer due diligence."
        }
    },
    {
        "company_information": {
            "legal_name": "Healthy Habits Fitness Studio",
            "dba_name": "Healthy Habits Fitness",
            "company_type": "Sole Proprietorship",
            "industry": "Health & Wellness",
            "industry_description": "Providing fitness classes, personal training, and wellness programs.",
            "website": "https://www.healthyhabitsfitness.com",
            "date_founded": "2021-11-12"
        },
        "business_details": {
            "expected_transaction_volume": 400,
            "average_transaction_value": 100,
            "business_model_description": "Membership fees, class packages, and personal training sessions."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Daniel Garcia",
                "email": "daniel.garcia@healthyhabitsfitness.com",
                "phone_number": "+1-555-789-0123"
            },
            "business_address": {
                "street_address": "525 Oakwood Ln",
                "city": "Miami",
                "state": "FL",
                "postal_code": "33133",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "09-8765432",
            "aml_kyc_policy": "We comply with applicable AML/KYC regulations and have procedures for customer identification and risk assessment."
        }
    },
    {
        "company_information": {
            "legal_name": "Pet Care Services Ltd.",
            "dba_name": "Pet Care Services",
            "company_type": "Corporation",
            "industry": "Pet Services",
            "industry_description": "Offering dog walking, pet sitting, and grooming services.",
            "website": "https://www.petcareservices.com",
            "date_founded": "2019-02-27"
        },
        "business_details": {
            "expected_transaction_volume": 250,
            "average_transaction_value": 50,
            "business_model_description": "Hourly rates for dog walking and pet sitting, and package pricing for grooming services."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Ashley Rodriguez",
                "email": "ashley.rodriguez@petcareservices.com",
                "phone_number": "+1-555-890-1234"
            },
            "business_address": {
                "street_address": "616 Willow Rd",
                "city": "Chicago",
                "state": "IL",
                "postal_code": "60606",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "87-6543210",
            "aml_kyc_policy": "We adhere to relevant AML/KYC regulations and have a process for customer verification."
        }
    },
    {
        "company_information": {
            "legal_name": "Home Improvement Solutions Co.",
            "dba_name": "Home Improvement Solutions",
            "company_type": "Partnership",
            "industry": "Construction & Home Improvement",
            "industry_description": "Providing home renovation, remodeling, and repair services.",
            "website": "https://www.homeimprovementsolutions.com",
            "date_founded": "2020-08-19"
        },
        "business_details": {
            "expected_transaction_volume": 150,
            "average_transaction_value": 7500,
            "business_model_description": "Project-based contracts with payment schedules tied to completion milestones."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Brian Williams",
                "email": "brian.williams@homeimprovementsolutions.com",
                "phone_number": "+1-555-901-2345"
            },
            "business_address": {
                "street_address": "707 Pinewood Dr",
                "city": "Atlanta",
                "state": "GA",
                "postal_code": "30303",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "76-5432109",
            "aml_kyc_policy": "We comply with AML/KYC regulations and have procedures for customer identification and due diligence."
        }
    },
    {
        "company_information": {
            "legal_name": "Fashion Boutique LLC",
            "dba_name": "Fashion Boutique",
            "company_type": "LLC",
            "industry": "Retail & Apparel",
            "industry_description": "Selling trendy clothing, accessories, and footwear for women.",
            "website": "https://www.fashionboutique.com",
            "date_founded": "2023-03-08"
        },
        "business_details": {
            "expected_transaction_volume": 600,
            "average_transaction_value": 120,
            "business_model_description": "Retail sales through a physical store and online e-commerce platform."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Jessica Brown",
                "email": "jessica.brown@fashionboutique.com",
                "phone_number": "+1-555-012-3456"
            },
            "business_address": {
                "street_address": "404 Oak St",
                "city": "Los Angeles",
                "state": "CA",
                "postal_code": "90001",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "65-4321098",
            "aml_kyc_policy": "We have established AML/KYC policies and procedures to mitigate financial crime risks."
        }
    },
    {
        "company_information": {
            "legal_name": "Acme Financial Technologies Inc.",
            "dba_name": "Acme FinTech",
            "company_type": "Corporation",
            "industry": "Financial Technology",
            "industry_description": "Develops payment processing software for e-commerce businesses.",
            "website": "https://www.acmefintech.com",
            "date_founded": "2019-04-01"
        },
        "business_details": {
            "expected_transaction_volume": 10000,
            "average_transaction_value": 50,
            "business_model_description": "Software as a Service (SaaS) model with monthly subscription fees based on transaction volume."
        },
        "contact_information": {
            "primary_contact": {
                "name": "John Smith",
                "email": "john.smith@acmefintech.com",
                "phone_number": "555-123-4567"
            },
            "business_address": {
                "street_address": "123 Main St",
                "city": "San Francisco",
                "state": "CA",
                "postal_code": "94105",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "12-3456789",
            "aml_kyc_policy": "We have a robust AML/KYC policy that includes customer identity verification, transaction monitoring, and risk assessment."
        }
    },
    {
        "company_information": {
            "legal_name": "Zenith Payments LLC",
            "dba_name": "ZenithPay",
            "company_type": "LLC",
            "industry": "Financial Services",
            "industry_description": "Provides mobile payment solutions for small businesses.",
            "website": "https://www.zenithpay.com",
            "date_founded": "2021-08-15"
        },
        "business_details": {
            "expected_transaction_volume": 5000,
            "average_transaction_value": 20,
            "business_model_description": "Charges a per-transaction fee for payment processing."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Jane Doe",
                "email": "jane.doe@zenithpay.com",
                "phone_number": "555-234-5678"
            },
            "business_address": {
                "street_address": "456 Oak Ave",
                "city": "Los Angeles",
                "state": "CA",
                "postal_code": "90001",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "98-7654321",
            "aml_kyc_policy": "We adhere to strict AML/KYC regulations and conduct thorough due diligence on all customers."
        }
    },
    {
        "company_information": {
            "legal_name": "Nova Financial Group Inc.",
            "dba_name": "NovaFin",
            "company_type": "Corporation",
            "industry": "Investment Management",
            "industry_description": "Offers robo-advisory services for automated investment portfolios.",
            "website": "https://www.novafin.com",
            "date_founded": "2017-03-10"
        },
        "business_details": {
            "expected_transaction_volume": 2000,
            "average_transaction_value": 1000,
            "business_model_description": "Charges a percentage fee based on assets under management."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Robert Lee",
                "email": "robert.lee@novafin.com",
                "phone_number": "555-345-6789"
            },
            "business_address": {
                "street_address": "789 Pine St",
                "city": "New York",
                "state": "NY",
                "postal_code": "10001",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "56-7890123",
            "aml_kyc_policy": "We maintain a comprehensive AML/KYC program to prevent financial crimes and ensure regulatory compliance."
        }
    },
    {
        "company_information": {
            "legal_name": "Quantum Solutions LLC",
            "dba_name": "QuantumSol",
            "company_type": "LLC",
            "industry": "Software Development",
            "industry_description": "Develops custom software solutions for financial institutions.",
            "website": "https://www.quantumsol.com",
            "date_founded": "2020-06-22"
        },
        "business_details": {
            "expected_transaction_volume": 500,
            "average_transaction_value": 10000,
            "business_model_description": "Project-based billing for software development services."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Maria Garcia",
                "email": "maria.garcia@quantumsol.com",
                "phone_number": "555-456-7890"
            },
            "business_address": {
                "street_address": "312 Cedar St",
                "city": "Chicago",
                "state": "IL",
                "postal_code": "60606",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "45-6789012",
            "aml_kyc_policy": "Our AML/KYC policy is designed to mitigate risks and comply with all applicable regulations."
        }
    },
    {
        "company_information": {
            "legal_name": "Apex Capital Partners LLC",
            "dba_name": "ApexCap",
            "company_type": "LLC",
            "industry": "Venture Capital",
            "industry_description": "Invests in early-stage fintech startups.",
            "website": "https://www.apexcap.com",
            "date_founded": "2018-09-18"
        },
        "business_details": {
            "expected_transaction_volume": 100,
            "average_transaction_value": 500000,
            "business_model_description": "Invests capital in exchange for equity ownership in startups."
        },
        "contact_information": {
            "primary_contact": {
                "name": "David Kim",
                "email": "david.kim@apexcap.com",
                "phone_number": "555-567-8901"
            },
            "business_address": {
                "street_address": "214 Birch St",
                "city": "Boston",
                "state": "MA",
                "postal_code": "02110",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "34-5678901",
            "aml_kyc_policy": "We have stringent AML/KYC procedures to verify the identity of investors and the source of funds."
        }
    },
    {
        "company_information": {
            "legal_name": "Veritas Data Security Inc.",
            "dba_name": "VeritasSec",
            "company_type": "Corporation",
            "industry": "Cybersecurity",
            "industry_description": "Provides data security solutions for financial institutions.",
            "website": "https://www.veritassec.com",
            "date_founded": "2016-02-05"
        },
        "business_details": {
            "expected_transaction_volume": 200,
            "average_transaction_value": 25000,
            "business_model_description": "Sells software licenses and provides security consulting services."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Linda Chen",
                "email": "linda.chen@veritassec.com",
                "phone_number": "555-678-9012"
            },
            "business_address": {
                "street_address": "987 Oakwood Lane",
                "city": "Austin",
                "state": "TX",
                "postal_code": "78701",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "23-4567890",
            "aml_kyc_policy": "We comply with all relevant data security and privacy regulations."
        }
    },
    {
        "company_information": {
            "legal_name": "Phoenix Insurance Group Inc.",
            "dba_name": "PhoenixInsure",
            "company_type": "Corporation",
            "industry": "Insurance",
            "industry_description": "Offers insurance products for businesses and individuals.",
            "website": "https://www.phoenixinsure.com",
            "date_founded": "2015-07-12"
        },
        "business_details": {
            "expected_transaction_volume": 1000,
            "average_transaction_value": 1500,
            "business_model_description": "Collects premiums and pays out claims."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Michael Brown",
                "email": "michael.brown@phoenixinsure.com",
                "phone_number": "555-789-0123"
            },
            "business_address": {
                "street_address": "654 Maple St",
                "city": "Seattle",
                "state": "WA",
                "postal_code": "98101",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "09-8765432",
            "aml_kyc_policy": "We have a comprehensive AML/KYC program to prevent fraud and money laundering."
        }
    },
    {
        "company_information": {
            "legal_name": "Global Payment Solutions Ltd.",
            "dba_name": "GlobalPay",
            "company_type": "Corporation",
            "industry": "Financial Technology",
            "industry_description": "Provides cross-border payment processing services.",
            "website": "https://www.globalpay.com",
            "date_founded": "2014-11-25"
        },
        "business_details": {
            "expected_transaction_volume": 5000,
            "average_transaction_value": 1000,
            "business_model_description": "Charges a fee for currency exchange and international payment processing."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Emily Wilson",
                "email": "emily.wilson@globalpay.com",
                "phone_number": "555-890-1234"
            },
            "business_address": {
                "street_address": "321 Pinewood Ave",
                "city": "London",
                "state": "",
                "postal_code": "EC1A 1AA",
                "country": "UK"
            }
        },
        "compliance_information": {
            "ein": "GB123456789",
            "aml_kyc_policy": "We follow strict AML/KYC protocols to comply with international regulations."
        }
    },
    {
        "company_information": {
            "legal_name": "Blockchain Technologies Corp.",
            "dba_name": "BlockTech",
            "company_type": "Corporation",
            "industry": "Blockchain",
            "industry_description": "Develops blockchain-based solutions for financial services.",
            "website": "https://www.blocktech.com",
            "date_founded": "2022-01-19"
        },
        "business_details": {
            "expected_transaction_volume": 1000,
            "average_transaction_value": 500,
            "business_model_description": "Provides blockchain development and consulting services."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Daniel Rodriguez",
                "email": "daniel.rodriguez@blocktech.com",
                "phone_number": "555-901-2345"
            },
            "business_address": {
                "street_address": "1010 Blockchain Blvd",
                "city": "Miami",
                "state": "FL",
                "postal_code": "33131",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "78-9012345",
            "aml_kyc_policy": "We are committed to building secure and compliant blockchain solutions."
        }
    },
    {
        "company_information": {
            "legal_name": "AI Finance Solutions Inc.",
            "dba_name": "AIFin",
            "company_type": "Corporation",
            "industry": "Artificial Intelligence",
            "industry_description": "Develops AI-powered financial forecasting tools.",
            "website": "https://www.aifin.com",
            "date_founded": "2023-05-08"
        },
        "business_details": {
            "expected_transaction_volume": 500,
            "average_transaction_value": 2000,
            "business_model_description": "Subscription-based access to AI financial forecasting software."
        },
        "contact_information": {
            "primary_contact": {
                "name": "Sophia Patel",
                "email": "sophia.patel@aifin.com",
                "phone_number": "555-012-3456"
            },
            "business_address": {
                "street_address": "404 Not Found Ave",
                "city": "Palo Alto",
                "state": "CA",
                "postal_code": "94301",
                "country": "USA"
            }
        },
        "compliance_information": {
            "ein": "67-8901234",
            "aml_kyc_policy": "We use AI to enhance our AML/KYC capabilities and ensure compliance."
        }
    }
]
"""

In [ ]:
#Load each entry into an array
import json
docs = []
loaded_json = json.loads(JSON_DOC)
for doc in loaded_json:
  docs.append(doc)

In [ ]:
#Verify the docs are a loaded
print(docs[0])

# Prompt Preprocessing

In [ ]:
#In this section we define the prompt, notice that the document will be included as part of the format
prompt_template_beggining = """
You are an expert compliance analyst.

Your task is to extract information from the sign up forms obtained.

1. Carefully analyze the document provided as context.
2. Write an intro paragraph for the company so the senior compliance analyst can quickly understand who the customer is.
3. Following the guidance for approved businesses add a paragraph after the intro explaining why the business is supportable or not.
4. Add another paragraph below explaining where the company is.
5. Always think step by step.

<supportable_businesses>
Summary of Prohibited & Restricted Businesses

This document outlines the types of businesses and activities that are generally not allowed or require prior approval to use payment processing services. These restrictions exist due to legal requirements, financial partner rules, and compliance and risk management policies of payment processors.

Types of Businesses:

High-Risk Jurisdictions and Persons: Businesses operating in or dealing with sanctioned countries and individuals.
Prohibited Businesses: Activities involving illegal products or services, adult content, certain financial services, marijuana, unfair practices, weapons, etc.
Restricted Businesses: Activities requiring prior written approval from the payment processor, such as CBD, financial products, pharmaceuticals, non-fiat currency, and more.
Jurisdiction-Specific Prohibited Businesses: Additional restrictions based on specific countries, such as India, Japan, Malaysia, Mexico, Singapore, Thailand, UAE, and the US.

Prohibited Uses of Payment Processing Services:

Misrepresentation of identity or business.
Facilitating transactions for undisclosed merchants.
Using the service as a virtual terminal.
Processing without actual goods or services.
Evasion of chargeback monitoring.
Sharing cardholder information.
Misuse of intellectual property.

Prohibitions for Card Issuing Products:
Consumer use for personal expenses.
International use outside of the registered jurisdiction.
Lending without proper licensing and approval.
Abusive practices like free trial abuse and scalping.
Non-compliance with marketing and user experience guidelines.
Inactivity for 12 consecutive months.
Incorrect integration type for employee/contractor cards.

Business Analyst Recommendations:
Based on this information, a business analyst should pay close attention to the following types of businesses:
Financial Services: This is a broad category with many restrictions and nuances. It's crucial to understand the specific requirements for lending, crowdfunding, money transmission, etc.
Regulated Industries: Industries like CBD, pharmaceuticals, and tobacco require careful consideration of compliance and legal aspects.
High-Risk Businesses: Businesses prone to fraud or abuse, like travel reservation services and multi-level marketing, need thorough risk assessments.
Emerging Technologies: Activities involving cryptocurrency, NFTs, and other new technologies should be evaluated based on current regulations and the payment processor's policies.
Jurisdiction-Specific Restrictions: Businesses operating in or targeting specific countries need to be aware of additional prohibitions and requirements.
Additional Considerations:
Business Model: Restrictions often depend on the specific business model and how the service is used.
Compliance: Understanding and adhering to relevant regulations is essential to avoid account closure or other consequences.
Risk Management: Businesses should have robust risk management practices to mitigate potential financial and legal risks.
Transparency: Maintaining clear and accurate information about the business and its activities is crucial for approval and continued use of payment processing services.
By carefully considering these factors, a business analyst can help ensure that businesses comply with the payment processor's policies and operate successfully within the platform.
</supportable_businesses>
<document>
"""
prompt_template_end="""
</document>

Response:

"""

# Inference

In [ ]:
import time

summaries = []
for doc in docs:
  result = generate(prompt_template_beggining,str(doc),prompt_template_end)
  #sleep 10 seconds to not exceed 6 QPM
  time.sleep(10)
  summaries.append(result.text)
  print(result.text)

# Result Postprocessing

In [ ]:
#In this section we will add the summaries to the original documents, afterwards the results can be stored in a database.

for  counter, doc in enumerate(docs):
  doc['summary'] = summaries[counter]

print(docs)

# Result Presentation